# Implementando uma LSTM no Sistema de Lorenz

## 1. Importando as bibliotecas necessárias

### 1.1 Bibliotecas gerais

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 125

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.style.use('dark_background')

### 1.2 Bibliotecas para Redes Neurais

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import backend as keras_backend

In [3]:
from sklearn.model_selection import GridSearchCV

In [4]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

### 1.3 Bibliotecas dos sistemas caóticos

In [5]:
import sys 
sys.path.insert(0, '../../../scripts')

import timeseries as times
import lorenzsystem as lsys

## 2. Gerando a série temporal do sistema de Lorenz

Os valores para os parâmetros das equações serão os "básicos" apresentados no *paper* original.

In [6]:
t_inicial = 0
t_final = 50
dt = 0.01

In [7]:
estado_inicial = np.array([0.1, 0, 0])

In [8]:
sis_lorenz = lsys.SistemaLorenz(estado_inicial, dt=dt)

In [9]:
solucoes, instantes_temporais = sis_lorenz.calcular(t_inicial = t_inicial, t_final = t_final)

In [10]:
x = solucoes[:, 0]
y = solucoes[:, 1]
z = solucoes[:, 2]

In [11]:
fig, ax = plt.subplots()
ax.set_title("Série temporal em $\hat{x}$ de 0 a 50 segundos do Sistema de Lorenz\n utilizando $\sigma = 10$, " + r"$\beta =\frac{8}{3}$, " + r"$\rho=28$, com " + "$x(0) =$ " + str(estado_inicial[0]) + ", $y(0) = $ " + str(estado_inicial[1]) + " e $z(0) =$ " + str(estado_inicial[2]))
ax.plot(instantes_temporais, x, color='DodgerBlue', linewidth = 0.9)

ax.set_ylabel('$x(t)$')
ax.set_xlabel('$t$')
ax.set_xlim(0,50)
ax.grid(True)

fig.tight_layout()
sns.despine()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [12]:
K = 4
L = 3
tam_teste = 0.15
tam_val = 0.1

In [13]:
serie_temporal = times.SerieTemporal(x, K=K, L=L)

In [15]:
X_treino, X_teste, X_val, y_treino, y_teste, y_val = serie_temporal.dividir_treino_teste_validacao(tam_teste, tam_val)

## 3. Definindo o modelo LSTM, treinando e realizando a predição

### 3.1 Definindo a função para criar a LSTM

In [16]:
def criar_modelo(activation='selu', init_mode='lecun_normal', n_neurons=30, learning_rate=0.001, name="LSTM-basica"):
    
    model = keras.Sequential(name=name)
    model.add(keras.Input(shape=(K,1)))
    model.add(keras.layers.LSTM(n_neurons, activation=activation, kernel_initializer=init_mode, name="camada_lstm"))
    model.add(keras.layers.Dense(1, activation='linear', name="camada_de_saida"))
    
    model_optimizer = keras.optimizers.Nadam()
    model_optimizer.learning_rate.assign(learning_rate)
    
    model.compile(
        optimizer = model_optimizer,
        loss = 'mse')
    
    model.build()
    return model

### 3.2 Pré-processamento nos dados de entrada

Dimensões dos vetores de entrada:

In [17]:
len_treino = X_treino.shape[0]
len_teste = X_teste.shape[0]
len_val = X_val.shape[0]

Ajustando os vetores para a abordagem *Many-to-One*:

In [18]:
X_treino = np.reshape(X_treino,(len_treino, K, 1))
X_teste = np.reshape(X_teste,(len_teste, K, 1))
X_val = np.reshape(X_val,(len_val, K, 1))

### 3.3 Treino com o modelo

In [19]:
model = criar_modelo()

In [20]:
model.summary()

Model: "LSTM-basica"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
camada_lstm (LSTM)           (None, 30)                3840      
_________________________________________________________________
camada_de_saida (Dense)      (None, 1)                 31        
Total params: 3,871
Trainable params: 3,871
Non-trainable params: 0
_________________________________________________________________


In [21]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss')

In [22]:
batch_size = 32

In [23]:
history = model.fit(X_treino, y_treino, epochs=100,
                            callbacks=early_stopping, validation_data=(X_val, y_val),
                            batch_size=batch_size, verbose=1)
treinamento = pd.DataFrame(history.history)

Epoch 1/100
120/120 [==============================] - 2s 6ms/step - loss: 47.0077 - val_loss: 1.8275
Epoch 2/100
120/120 [==============================] - 0s 4ms/step - loss: 0.9263 - val_loss: 0.3241
Epoch 3/100
120/120 [==============================] - 0s 4ms/step - loss: 0.2678 - val_loss: 0.0845
Epoch 4/100
120/120 [==============================] - 0s 4ms/step - loss: 0.1109 - val_loss: 0.0250
Epoch 5/100
120/120 [==============================] - 0s 4ms/step - loss: 0.0311 - val_loss: 0.0111
Epoch 6/100
120/120 [==============================] - 0s 4ms/step - loss: 0.0637 - val_loss: 0.0113
Epoch 7/100
120/120 [==============================] - 0s 4ms/step - loss: 0.0168 - val_loss: 0.0411
Epoch 8/100
120/120 [==============================] - 0s 4ms/step - loss: 0.0284 - val_loss: 0.0053
Epoch 9/100
120/120 [==============================] - 0s 4ms/step - loss: 0.0168 - val_loss: 0.1800
Epoch 10/100
120/120 [==============================] - 0s 4ms/step - loss: 0.0470 - val_l

## Teste com o melhor modelo

In [24]:
y_pred = model.predict(X_teste)

In [25]:
len(instantes_temporais[len(instantes_temporais)-int(len(instantes_temporais)*tam_teste):,])

750

### Gráfico

In [27]:
fig, ax = plt.subplots()
ax.plot(instantes_temporais[len(instantes_temporais)-int(len(instantes_temporais)*tam_teste):,], y_teste, color='DodgerBlue', label='Valor real')
ax.plot(instantes_temporais[len(instantes_temporais)-int(len(instantes_temporais)*tam_teste):,], y_pred, color='Silver', label='LSTM')

ax.set_title("Comparação da predição da LSTM com o valor real do sistema de Lorenz")
ax.set_ylabel('$x(t)$')
ax.set_xlabel('$t$')
ax.set_xlim(42.5, 50)
    
ax.grid(True)
sns.despine()
ax.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Erro Quadrático Médio

In [28]:
from sklearn.metrics import mean_squared_error

In [29]:
mse = mean_squared_error(y_teste, y_pred)
print("MSE = " + str(mse))

MSE = 0.0047299637797572
